In [1]:
import pandas as pd
import numpy as np
import os

from scipy.signal import convolve2d as conv2d
import time
import cv2
import nibabel as nb
# from imea.measure_2d.micro import fractal_dimension_boxcounting as fract_dim

import warnings
warnings.filterwarnings('ignore')

In [2]:
def lac1(x):
    bin_ = np.zeros((x.shape[0], x.shape[1]))
    bin_[x!=0] = 1
    bin_ = bin_.astype(np.uint8)
    
    contours = cv2.findContours(image = bin_, mode = cv2.RETR_TREE, method = cv2.CHAIN_APPROX_SIMPLE)[0]
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    
    if len(contours) == 0:
        return 0
    
    c_0 = contours[0]               #Get the 4 points of the bounding rectangle
    xx, yy, w, h = cv2.boundingRect(c_0)
    X  =  x[yy:yy+h, xx:xx+w]

    box_size =  [i for i in range(2, 240) if ((i)**2 < (0.45)*(X.shape[0]* X.shape[1])) and i <= min(X.shape[0], X.shape[1])]

    LAMBDA = []
    
    XX = np.zeros((X.shape[0], X.shape[1]))
    
    XX[X==1] = 1
  
    for box in box_size:
        count = []
        count, edge= np.histogram(np.ravel(conv2d(XX, np.ones((box, box)), mode = 'valid')), bins = [i for i in range(0, (box**2) + 2)])#(box**2) + 1)
        q = count/(XX.shape[0] - box + 1)**2
        s = np.array([i for i in range(0,box**2 + 1)])
        lam_bda =  sum((s**2)*q)/(sum(q*s))**2
        
     
        LAMBDA.append(lam_bda)
        
    return np.nanmean(LAMBDA)

In [3]:
def lac2(x):
    bin_ = np.zeros((x.shape[0], x.shape[1]))
    bin_[x!=0] = 1
    bin_ = bin_.astype(np.uint8)

    contours = cv2.findContours(image = bin_, mode = cv2.RETR_TREE, method = cv2.CHAIN_APPROX_SIMPLE)[0]
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    
    if len(contours) == 0:
        return 0
    
    c_0 = contours[0]               #Get the 4 points of the bounding rectangle
    xx, yy, w, h = cv2.boundingRect(c_0)
    X  =  x[yy:yy+h, xx:xx+w]

    box_size =  [i for i in range(2, 240) if ((i)**2 < (0.45)*(X.shape[0]* X.shape[1])) and i <= min(X.shape[0], X.shape[1])]

    LAMBDA = []

    XX = np.zeros((X.shape[0], X.shape[1]))
    XX[X==2] = 1
  
    for box in box_size:
        count = []
        count, edge= np.histogram(np.ravel(conv2d(XX, np.ones((box, box)), mode = 'valid')), bins = [i for i in range(0, (box**2) + 2)])#(box**2) + 1)
        q = count/(XX.shape[0] - box + 1)**2
        s = np.array([i for i in range(0,box**2 + 1)])
        lam_bda =  sum((s**2)*q)/(sum(q*s))**2


        LAMBDA.append(lam_bda)
        
    return np.nanmean(LAMBDA)

In [4]:
def lac4(x):
    bin_ = np.zeros((x.shape[0], x.shape[1]))
    bin_[x!=0] = 1
    bin_ = bin_.astype(np.uint8)

    contours = cv2.findContours(image = bin_, mode = cv2.RETR_TREE, method = cv2.CHAIN_APPROX_SIMPLE)[0]
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    
    if len(contours) == 0:
        return 0
    
    c_0 = contours[0]               #Get the 4 points of the bounding rectangle
    xx, yy, w, h = cv2.boundingRect(c_0)
    X  =  x[yy:yy+h, xx:xx+w]
    
    box_size =  [i for i in range(2, 240) if ((i)**2 < (0.45)*(X.shape[0]* X.shape[1])) and i <= min(X.shape[0], X.shape[1])]

    LAMBDA = []
    
    XX = np.zeros((X.shape[0], X.shape[1]))
    
    XX[X==4] = 1
  
    for box in box_size:
        count = []
        count, edge= np.histogram(np.ravel(conv2d(XX, np.ones((box, box)), mode = 'valid')), bins = [i for i in range(0, (box**2) + 2)])#(box**2) + 1)
        q = count/(XX.shape[0] - box + 1)**2
        s = np.array([i for i in range(0,box**2 + 1)])
        lam_bda =  sum((s**2)*q)/(sum(q*s))**2

        LAMBDA.append(lam_bda)
    # print(LAMBDA)
        
    return np.mean(LAMBDA)

In [5]:
def frac_lac(X):
    X = X.astype(np.uint8)


    X = np.delete(X, [i for i in range(0, 155) if np.sum(X[:,:,i].ravel()) < 15], 2)
    
    X_n = np.delete(X, [i for i in range(0, X.shape[-1]) if np.sum(X[:,:,i].ravel() == 1) < 12], 2)
    X_et = np.delete(X, [i for i in range(0, X.shape[-1]) if np.sum(X[:,:,i].ravel() == 4) < 6], 2)
    X_ed = np.delete(X, [i for i in range(0, X.shape[-1]) if np.sum(X[:,:,i].ravel() == 2) < 6], 2)
    

    X_n = [X_n[:,:,i] for i in range(0,X_n.shape[-1]) ]
    X_et = [X_et[:,:,i] for i in range(0,X_et.shape[-1]) ]
    X_ed = [X_ed[:,:,i] for i in range(0,X_ed.shape[-1]) ]
    
    nc = np.array([lac1(x) for x in X_n])
    ed = np.array([lac2(x) for x in X_ed])
    et = np.array([lac4(x) for x in X_et])

    n_mean =  np.nanmean(nc)
    n_med =  np.nanmedian(nc)
    
    ed_mean =  np.nanmean(ed)
    ed_med =  np.nanmedian(ed)
    
    et_mean =  np.nanmean(et)
    et_med =  np.nanmedian(et)

    return  np.array([n_mean, n_med, ed_mean, ed_med, et_mean, et_med])

In [6]:
gbm_lac = []
t0 = time.time()

with open('gbm_input.txt') as file:    
    for fl in file:
        try:
            gbm_lac.append(frac_lac(nb.load(r'{}'.format(fl.rstrip())).get_fdata()))
        except:
            continue
        
print('Runtime:', time.time() - t0 , 'seconds')  
x = np.array(gbm_lac)


case_id = []
with open('gbm_input.txt') as file:
    for fl in file:
        case_id.append(fl.split('\\')[5])    #Number is the position of subject code in tha path filename
        
print(case_id[0])    #to check what is being printed
        
        
gbm_lac_df = pd.DataFrame({'case_id':case_id,
                          'ncr_net_meanlac': x[:, 0], 'ncr_net_medlac': x[:, 1],
                         'ed_meanlac': x[:, 2], 'ed_medlac':x[:, 3],
                         'et_meanlac': x[:, 4], 'et_medlac': x[:, 5]})

# gbm_lac.to_csv('gbm_lac.csv')

Runtime: 1554.810474395752 seconds
TCGA-02-0006


In [7]:
lgg_lac = []
t0 = time.time()

with open('lgg_input.txt') as file:    
    for fl in file:
        try:
            lgg_lac.append(frac_lac(nb.load(r'{}'.format(fl.rstrip())).get_fdata()))
        except:
            continue
        
print('Runtime:', time.time() - t0 , 'seconds')  
x = np.array(lgg_lac)


case_id2 = []

with open('lgg_input.txt') as file:
    for fl in file:
        case_id2.append(fl.split('\\')[5])    #Number is the position of subject code in tha path filename
        
print(case_id2[0])    #to check what is being printed
        
        
lgg_lac_df = pd.DataFrame({'case_id':case_id2,
                          'ncr_net_meanlac': x[:, 0], 'ncr_net_medlac': x[:, 1],
                         'ed_meanlac': x[:, 2], 'ed_medlac':x[:, 3],
                         'et_meanlac': x[:, 4], 'et_medlac': x[:, 5]})

# lgg_lac.to_csv('lgg_lac.csv')

Runtime: 1075.1896278858185 seconds
TCGA-CS-4942


In [8]:
df = pd.concat([gbm_lac_df, lgg_lac_df])
df.to_csv('lac_merged.csv', index=False)

In [ ]:
import os
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

notebook_filename_list = ['3DLac_cor.ipynb','3DLac_sag.ipynb']

for notebook_filename in notebook_filename_list:
    before_files = set(os.listdir('.'))

    with open(notebook_filename) as f:
        notebook = nbformat.read(f, as_version=4)


    ep = ExecutePreprocessor(kernel_name='python3')

    try:
        ep.preprocess(notebook, {'metadata': {'path': './'}})
        print(f'{notebook_filename} executed successfully!')
    except Exception as e:
        print(f'Error executing {notebook_filename}: {e}')

    ## To search for files created after running the code
    after_files = {f: os.path.getmtime(f) for f in os.listdir('.') if f.endswith('.csv')}

    # Find new or modified CSV files
    new_or_modified_files = [f for f in after_files if f not in before_files or after_files[f] != before_files[f]]

    # Print the list of new or modified CSV files
    if new_or_modified_files:
        print('New or modified CSV files:')
        for file in new_or_modified_files:
            print(file)
    else:
        print('No new or modified CSV files.')

The above code snippet generates the following files: 'lac_cor_merged.csv' and 'lac_sag_merged.csv'

In [10]:
lac_df1 = pd.read_csv('lac_merged.csv')
lac_df2 = pd.read_csv('lac_cor_merged.csv')
lac_df3 = pd.read_csv('lac_sag_merged.csv')

In [20]:
lac_df2.rename(columns={'ID': 'case_id'}, inplace=True)
lac_df3.rename(columns={'ID': 'case_id'}, inplace=True)

In [21]:
lac_temp = pd.merge(lac_df1, lac_df2, on='case_id', how='inner')
lac_merged = pd.merge(lac_temp, lac_df3, on='case_id', how='inner')

In [22]:
selected_columns = ['ncr_net_meanlac', 'ncr_net_meanlac_cor', 'ncr_net_meanlac_sag']
mean_values = lac_merged[selected_columns].mean(axis=1)
lac_merged['ncr_net_mean3dlac'] = mean_values

selected_columns2 = ['et_meanlac', 'et_meanlac_cor', 'et_meanlac_sag']
mean_values2 = lac_merged[selected_columns2].mean(axis=1)
lac_merged['et_mean3dlac'] = mean_values2

selected_columns3 = ['ed_meanlac', 'ed_meanlac_cor', 'ed_meanlac_sag']
mean_values3 = lac_merged[selected_columns3].mean(axis=1)
lac_merged['ed_mean3dlac'] = mean_values3

In [23]:
lac_merged.to_csv('3DLac.csv', index=False)

### Merging Fractal Dimension and Lacunarity

In [27]:
frac = pd.read_csv('3dFrac.csv')
lac = pd.read_csv('3dLac.csv')\

fraclac = pd.merge(frac, lac, on='case_id', how='inner')
fraclac.rename(columns={'case_id': 'ID'}, inplace=True)
fraclac.to_csv('fractal_lac_data.csv', index=False)